In [316]:
import pandas as pd
import numpy as np

In [317]:
benchmark_df = pd.read_csv("kse100.csv", usecols=["Return"])
benchmark_returns = np.array(benchmark_df["Return"])
print(benchmark_returns)

[ 0.0472 -0.0131 -0.0168  0.0296  0.0654  0.0601 -0.0112 -0.0278 -0.0073
  0.0821 -0.0113 -0.0064  0.0077 -0.0531  0.0286 -0.0241 -0.0106  0.0175
 -0.0201  0.0105  0.0071 -0.048  -0.0357 -0.0335  0.0548 -0.0289  0.0033
  0.0263 -0.0455  0.0062 -0.004  -0.0126  0.0395 -0.006   0.003   0.1588
 -0.0631  0.0273  0.123   0.1659  0.0317 -0.0076  0.0419  0.0376  0.0612
  0.0672  0.0338 -0.0071  0.0077  0.0335  0.0968  0.1393  0.1359 -0.0076
 -0.0088  0.0402 -0.055   0.0751  0.0496  0.1096]


In [318]:
mean_weights = [0.5, 0.5]
stag_weights = [0.9, 0.1]
growth_weights = [0.1, 0.9]

def fitness(returns, periods=[(24, 36), (48, 60)], weights=mean_weights):
    """
        For each period calculates average returns divided by standard deviation of returns.
        Then returns the average of the two periods.
    """
    period_fitness = []
    for start, end in periods:
        period_returns = returns[start:end]
        avg_return = np.mean(period_returns)
        std_return = np.std(period_returns)
        period_fitness.append(avg_return / std_return) # if std_return != 0 else 0
    return (period_fitness[0] * weights[0]) + (period_fitness[1] * weights[1])
print(fitness(benchmark_returns))

0.5969457198568304


In [319]:
def total_money(returns, periods=[(24, 36), (48, 60)], weights=mean_weights):
    """
        Calculates the investment growth over each period and returns the weighted sum of the two periods.
    """
    totals = []
    for start, end in periods:
        period_returns = returns[start:end]
        total = 1.0
        for r in period_returns:
            total *= (1.0 + r)
        totals.append(total)
    return (totals[0] * weights[0]) + (totals[1] * weights[1])
print(total_money(benchmark_returns))

1.4930503760657379


In [320]:
def total_risk(returns, periods=[(24, 36), (48, 60)], weights=mean_weights):
    """
        Calculates the total risk (standard deviation of returns) over each period and returns the weighted sum of the two periods.
    """
    risks = []
    for start, end in periods:
        period_returns = returns[start:end]
        risk = np.var(period_returns)
        risks.append(risk)
    return np.sqrt((risks[0] * weights[0]) + (risks[1] * weights[1]))
print(total_risk(benchmark_returns))

0.05482661243005586


In [321]:
def get_stock_returns(file_path):
    """
        The returns csv has a stock symbol column and 60 columns for returns.
        This function reads the csv and returns a dframe with stock symbols as index.
    """
    df = pd.read_csv(file_path)
    df.set_index("stock", inplace=True)
    return df
stock_returns_df = get_stock_returns("returns.csv")
returns_array = np.array(stock_returns_df)

In [322]:
# Get index of a stock symbol from the dataframe
stock_symbol = "JVDC"
stock_index = stock_returns_df.index.get_loc(stock_symbol)
# print(f"Index of {stock_symbol}: {stock_index}")
print(returns_array[stock_index])

[ 0.01858913 -0.10528779 -0.06276151  0.02287946  0.06819422  0.07354443
  0.14652712  0.30082988 -0.09665072  0.17655367  0.05552221 -0.04577765
  0.41716329  0.04941127 -0.02825085  0.07670103 -0.01761777 -0.04132554
 -0.10329402  0.11020408 -0.01940359 -0.06269527 -0.06666667  0.03571429
  0.14482759  0.02409639  0.06078431 -0.05914972 -0.03732809 -0.00102041
  0.00102145 -0.06122449 -0.04347826 -0.11318182 -0.01973347  0.04444444
 -0.06132666  0.11946667 -0.13935207  0.02435649  0.0094569  -0.04175589
 -0.06145251 -0.0327381   0.03076923  0.08597015 -0.03628367  0.14632059
  0.03508335  0.00673077 -0.06876791  0.45717949  0.05349287  0.08819108
 -0.13046815  0.14651368 -0.09160893  0.10627119 -0.0140953   0.17031857]


In [323]:
portfolios_df = pd.read_csv("portfolios.csv") # Portfolio, Stock, Weight
portfolios_df.head()

,Portfolio,Stock,Weight
0,roulette_unsorted_stag,JVDC,0.202973
1,roulette_unsorted_stag,JDWS,0.188741
2,roulette_unsorted_stag,SCBPL,0.112246
3,roulette_unsorted_stag,SAZEW,0.106154
4,roulette_unsorted_stag,PTC,0.054597


In [324]:
def get_portfolio_returns(portfolio_id):
    """
        Given a portfolio id, returns the weighted returns of the portfolio.
    """
    portfolio = portfolios_df[portfolios_df["Portfolio"] == portfolio_id]
    portfolio_returns = np.zeros(60)
    for _, row in portfolio.iterrows():
        stock_symbol = row["Stock"]
        weight = row["Weight"]
        stock_index = stock_returns_df.index.get_loc(stock_symbol)
        stock_returns = returns_array[stock_index]
        portfolio_returns += weight * stock_returns
    return portfolio_returns

In [325]:
get_portfolio_returns("roulette_unsorted_stag")

array([ 0.04427284, -0.02474135,  0.02267116,  0.01172384,  0.04754346,
        0.03667274,  0.03650854,  0.03096718, -0.01664114,  0.07446628,
        0.01511889,  0.01654286,  0.07573206, -0.0282499 ,  0.03057083,
       -0.01975887, -0.01831499,  0.0533708 , -0.06018662,  0.02344289,
       -0.01382979, -0.03181775, -0.07360286,  0.02634061,  0.05854907,
       -0.01585766,  0.0868172 ,  0.02743129, -0.03087456, -0.02518057,
       -0.00125753, -0.04727987,  0.02878813, -0.01099423, -0.02035432,
        0.13428055, -0.03176553,  0.07332399,  0.08845411,  0.07521275,
        0.07313379,  0.04463875,  0.0547893 ,  0.07183239,  0.12540987,
        0.09534796,  0.00245901,  0.06395517,  0.00220275, -0.01736943,
        0.0514079 ,  0.14185236,  0.13281898,  0.04669121, -0.01970383,
        0.02614577, -0.02808199,  0.06571775,  0.00697066,  0.10322099])

In [326]:
mean_weights = [0.5, 0.5]
stag_weights = [0.95, 0.05]
growth_weights = [0.05, 0.95]

In [327]:
def summarize_portfolios():
    """
        Summarizes all portfolios by calculating their fitness.
    """
    portfolio_ids = portfolios_df["Portfolio"].unique()
    summary = []
    for pid in portfolio_ids:
        returns = get_portfolio_returns(pid)
        # Mean
        fit = fitness(returns)
        tm = total_money(returns)
        tr = total_risk(returns)
        leaning = "Mean"
        summary.append((pid, fit, tm, tr, leaning))
        # Stag
        fit = fitness(returns, weights=stag_weights)
        tm = total_money(returns, weights=stag_weights)
        tr = total_risk(returns, weights=stag_weights)
        leaning = "Stag"
        summary.append((pid, fit, tm, tr, leaning))
        # Growth
        fit = fitness(returns, weights=growth_weights)
        tm = total_money(returns, weights=growth_weights)
        tr = total_risk(returns, weights=growth_weights)
        leaning = "Growth"
        summary.append((pid, fit, tm, tr, leaning))
    summary_df = pd.DataFrame(summary, columns=["Portfolio", "Fitness", "Total Money", "Total Risk", "Leaning"])
    return summary_df

In [328]:
# Find the top portfolio for each leaning by money
summary_df = summarize_portfolios()
summary_df.sort_values(by="Total Money", ascending=False, inplace=True)
top_mean = summary_df[summary_df["Leaning"] == "Mean"].iloc[0]
top_stag = summary_df[summary_df["Leaning"] == "Stag"].iloc[0]
top_growth = summary_df[summary_df["Leaning"] == "Growth"].iloc[0]
print("Mean:", top_mean.Portfolio, top_mean.Fitness, top_mean["Total Money"], top_mean["Total Risk"])
print("Stag:", top_stag.Portfolio, top_stag.Fitness, top_stag["Total Money"], top_stag["Total Risk"])
print("Growth:", top_growth.Portfolio, top_growth.Fitness, top_growth["Total Money"], top_growth["Total Risk"])

Mean: roulette_unsorted_growth 0.5311253819866123 1.405330521430784 0.05629481771674338
Stag: tournament_unsorted_growth 0.36875645438524374 1.228210358153302 0.050023994233833934
Growth: roulette_sorted_stag 0.8735149387261573 1.6731992716379098 0.05113017766196322


In [329]:
# Find the top portfolio for each leaning by fitness
summary_df.sort_values(by="Fitness", ascending=False, inplace=True)
top_mean = summary_df[summary_df["Leaning"] == "Mean"].iloc[0]
top_stag = summary_df[summary_df["Leaning"] == "Stag"].iloc[0]
top_growth = summary_df[summary_df["Leaning"] == "Growth"].iloc[0]
print("Mean:", top_mean.Portfolio, top_mean.Fitness, top_mean["Total Money"], top_mean["Total Risk"])
print("Stag:", top_stag.Portfolio, top_stag.Fitness, top_stag["Total Money"], top_stag["Total Risk"])
print("Growth:", top_growth.Portfolio, top_growth.Fitness, top_growth["Total Money"], top_growth["Total Risk"])

Mean: tournament_unsorted_growth 0.5636926418146831 1.366913916348612 0.048588832748663
Stag: tournament_unsorted_growth 0.36875645438524374 1.228210358153302 0.050023994233833934
Growth: roulette_sorted_stag 0.8735149387261573 1.6731992716379098 0.05113017766196322


In [330]:
# Find the top portfolio for each leaning by low-risk
summary_df.sort_values(by="Total Risk", ascending=True, inplace=True)
top_mean = summary_df[summary_df["Leaning"] == "Mean"].iloc[0]
top_stag = summary_df[summary_df["Leaning"] == "Stag"].iloc[0]
top_growth = summary_df[summary_df["Leaning"] == "Growth"].iloc[0]
print("Mean:", top_mean.Portfolio, top_mean.Fitness, top_mean["Total Money"], top_mean["Total Risk"])
print("Stag:", top_stag.Portfolio, top_stag.Fitness, top_stag["Total Money"], top_stag["Total Risk"])
print("Growth:", top_growth.Portfolio, top_growth.Fitness, top_growth["Total Money"], top_growth["Total Risk"])

Mean: tournament_sorted_stag 0.43907047160828744 1.2958760365988427 0.04328274946015426
Stag: tournament_sorted_growth 0.1444723975350181 1.0647614915928434 0.035924156542419185
Growth: tournament_unsorted_stag 0.5941077567927199 1.3168978115982972 0.042805459919694235


In [ ]:
# Compare with benchmark
for weights in [mean_weights, stag_weights, growth_weights]:
    print("Weights:", weights)
    print("Benchmark - Fitness:", fitness(benchmark_returns, weights=weights), 
          "Total Money:", total_money(benchmark_returns, weights=weights), 
          "Total Risk:", total_risk(benchmark_returns, weights=weights))

Weights: [0.5, 0.5]
Benchmark - Fitness: 0.5969457198568304 Total Money: 1.4930503760657379 Total Risk: 0.05482661243005586
Weights: [0.95, 0.05]
Benchmark - Fitness: 0.349726478711617 Total Money: 1.2260546658170037 Total Risk: 0.050859668997699496
Weights: [0.05, 0.95]
Benchmark - Fitness: 0.8441649610020437 Total Money: 1.7600460863144718 Total Risk: 0.05852528454057746
